<p style="text-align: center;"><span style="color: #ff0000;"><strong><span style="font-size: x-large;">
    ANEXO XXX: ALGORITMOS PKE</span></strong></span></p>

<p style="text-align: center;"><span style="color: black;"><strong><span style="font-size: x-large;">Realizado por:</span></strong></span></p>
<p style="text-align: center;"><span style="color: black;"><strong><span style="font-size: x-large;">Gabriel Vacaro Goytia</span></strong></span></p>
<p style="text-align: center;"><span style="color: black;"><strong><span style="font-size: x-large;">Ignacio Warleta Murcia</span></strong></span></p>

Este notebook contiene una implementación de los algoritmos fundamentales del esquema de cifrado post–cuántico Kyber, en particular el KyberKEM (Key Encapsulation Mechanism). Kyber es un sistema criptográfico basado en el problema de Redes Lattice y es uno de los candidatos más destacados para ser parte del estándar de criptografía post-cuántica propuesto por el NIST.

Organizamos el anexo según el siguiente índice:

# Índice

1. [Introducción](#1.-Introducción)
2. [Configuración previa](#2.-Configuracion-previa)
3. [Algoritmo de generacion de claves KEM](#3.-Algoritmo-de-generacion-de-claves-KEM)
4. [Algoritmo de encapsulado](#4.-Algoritmo-de-encapsulado)
5. [Algoritmo de desencapsulado](#5.-Algoritmo-de-desencapsulado)
6. [Ejemplo de uso](#6.-Ejemplo-de-uso)

---
# 1. Introducción






Este notebook cuenta con los 3 algoritmos relativos al KEM de Kyber–KEM:

- Generación de claves (Key Generation) <br>
- Encapsulado (Encapsulation) <br>
- Desencapsulado (Decapsulation) <br>

El propósito de esta implementación es comprender los detalles detrás de cada fase del esquema Kyber–KEM de manera didáctica, con especial énfasis en los aspectos técnicos que permiten asegurar la privacidad y la integridad de la comunicación en un entorno potencialmente afectado por computadoras cuánticas. <br>


---
# 2. Configuracion previa

A continuación, se muestra la configuración previa a ejecutar y los parámetros a definir. En este caso hemos optado por un primo $q$ relativamente pequeño, 743, dado su caracter didáctico, ya que, de esta manera se puede apreciar ligeramente el error que se produce al descifrar el mensaje. Por supuesto, el usuario es libre de cambiar este primo a su gusto y comprobar por su cuenta como se maneja el error. Además haremos uso de los algoritmos ya vistos relativos al PKE como son la generación de claves, el cifrado y descifrado. Cabe decir que se han realizado modificaciones al método implementado de cifrado para que sea capaz de soportar las semillas de mayor tamaño con las que trabajamos en encapsulación.

In [1]:
#MODULOS A IMPORTAR
import numpy as np
import hashlib

In [2]:
# Parámetros básicos
q = 743  # Un número primo pequeño típico en Kyber podría ser 3329
k = 3  # Tamaño del vector/matriz (varía según los estándares Kyber-512, 768, 1024)
mu_1 = 1.0  # Parámetro para la distribución de error más controlado (menor desviación estándar)

In [3]:
# Algoritmos PKE

# Funciones auxiliares
def generate_seed():
    return np.random.bytes(16)

def generate_matrix_A(seed, k, q):
    np.random.seed(int.from_bytes(seed, "big") % (2**32))
    return np.random.randint(0, q, size=(k, k))

def sample_error(mu, k, q, seed=None):
    if seed is not None:
        state = np.random.get_state()
        # Convertir a entero de 32 bits válido
        seed = seed % (2**32)  # <--- Añade esta línea
        np.random.seed(seed)
    error = np.round(np.random.normal(0, mu, size=(k, 1))).astype(int) % q
    if seed is not None:
        np.random.set_state(state)
    return error

# Algoritmo 1: Generación de Claves G'
def key_generation():
    seed_A = generate_seed()
    A = generate_matrix_A(seed_A, k, q)
    s = sample_error(mu_1, k, q)
    e = sample_error(mu_1, k, q)
    b = (A @ s + e) % q
    return (b, seed_A), s

# Algoritmo 2: Cifrado modificado para usar semilla
def encrypt(pk, m, seed=None):
    b, seed_A = pk
    if seed:
        hash_output = hashlib.sha3_256(seed).digest()
        r_seed = hash_output[:4]  # Usar solo 4 bytes (32 bits)
        e1_seed = hash_output[4:8]
        e2_seed = hash_output[8:12]
        r = sample_error(mu_1, k, q, int.from_bytes(r_seed, 'big'))
        e_1 = sample_error(mu_1, k, q, int.from_bytes(e1_seed, 'big'))
        e_2 = sample_error(mu_1, k, q, int.from_bytes(e2_seed, 'big'))
    else:
        r = sample_error(mu_1, k, q)
        e_1 = sample_error(mu_1, k, q)
        e_2 = sample_error(mu_1, k, q)
    
    A = generate_matrix_A(seed_A, k, q)
    A_T = A.T
    u = (A_T @ r + e_1) % q
    v = (b.T @ r + e_2 + m) % q
    return (u, v), r, e_1, e_2

# Algoritmo 3: Descifrado
def decrypt(sk, c):
    u, v = c
    s = sk
    return (v - (s.T @ u)) % q

---
# 3. Algoritmo de generacion de claves KEM

1. **Generación del vector $z$:**  
   Primeramente, generamos un vector $z$ de 256 bits aleatorios. Este valor se utiliza como «entropía» o «salto aleatorio» y es esencial para mantener la seguridad del sistema. Se usa en la clave secreta final $sk$, y la elección de 256 bits proporciona suficiente aleatoriedad para asegurar que la clave secreta sea impredecible.

2. **Llamada al algoritmo de generación de claves $G'$:**  
   En el siguiente paso, llamamos al algoritmo de generación de claves $G'$ descrito anteriormente. De este algoritmo, obtenemos la clave pública $pk$ y la clave secreta intermedia $sk'$, que solo se utilizará para generar la nueva clave secreta final $sk$.

3. **Construcción de la clave secreta final $sk$:**  
   En este paso, construimos la clave secreta final $sk$ concatenando varios componentes:  
   - La clave secreta intermedia $sk'$.  
   - La clave pública $pk$.  
   - El resultado de aplicar una función hash $H$ sobre la clave pública, $H(pk)$.  
   El propósito de aplicar la función hash sobre la clave pública es incluir un resumen de la misma en la clave secreta. Esto mejora la seguridad, ya que asegura que la clave secreta dependa de la clave pública, lo que ayuda a evitar ciertos tipos de ataques.

4. **Concatenación final y devolución de claves:**  
   Finalmente, concatenamos el vector $z$ y se devuelve la clave pública $pk$ junto con la clave secreta final $sk$.


In [4]:
# Algoritmo 4: Generación de Claves KEM
def key_gen_G():
    z = np.random.bytes(32)
    pk, sk_prime = key_generation()
    b, seed_A = pk
    sk_prime_bytes = sk_prime.tobytes()
    pk_bytes = b.tobytes() + seed_A
    H_pk = hashlib.sha3_256(pk_bytes).digest()
    sk = sk_prime_bytes + pk_bytes + H_pk + z
    return pk, sk

---
# 4. Algoritmo de encapsulado

1. **Generación del valor $m'$:**  
   El primer paso del algoritmo de encapsulado consiste en generar un valor $m'$ de 256 bits aleatorios. Luego, en el siguiente paso, se produce un valor $m$ al aplicarle a $m'$ una función hash $H$.  
   El propósito de este paso es generar una «clave derivada» a partir de los 256 bits aleatorios, pero con una forma adecuada para el cifrado, es decir, con una longitud estándar, lo cual se consigue gracias a la función hash $H$.

2. **Construcción de la clave compartida $K ̅$ y valor $r$:**  
   En el tercer paso, construimos una clave $K ̅$ y un valor $r$ concatenando $m$ y el hash de la clave pública, $H(pk)$.  
   - $K ̅$ es una clave compartida que se utilizará para la comunicación posterior.  
   - La concatenación de $m$ y $H(pk)$ garantiza que $K ̅$ dependa tanto del valor aleatorio inicial como de la clave pública del destinatario.  
   Esto asegura que el proceso de cifrado sea seguro y dependiente de la clave pública.

3. **Cifrado del mensaje ($c$):**  
   En el siguiente paso, se realiza el cifrado utilizando la clave pública y los valores $m$ y $r$ como parámetros de entrada.  
   El resultado de este paso es el mensaje cifrado $c$.

4. **Derivación de la clave compartida final $K$:**  
   Seguidamente, se realiza una función de derivación de claves (KDF, Key Derivation Function) para generar la clave compartida final $K$.  
   - Primero, se concatenan la clave $K ̅$ generada en el paso 2 y el hash de $c$.  
   - La función KDF toma esta concatenación y produce la clave derivada final $K$.  
   El uso de KDF asegura que la clave derivada final $K$ tenga una longitud adecuada y sea segura para ser utilizada como una clave secreta compartida entre el emisor y el receptor.

5. **Devolución de los resultados:**  
   Finalmente, el algoritmo devuelve el mensaje cifrado $c$ y la clave compartida $K$.


In [6]:
# Algoritmo 5: Encapsulado KEM
def encapsulate(pk):
    m_prime = np.random.bytes(32)
    m = hashlib.sha3_256(m_prime).digest()
    b, seed_A = pk
    pk_bytes = b.tobytes() + seed_A
    H_pk = hashlib.sha3_256(pk_bytes).digest()
    m_Hpk = m + H_pk
    K_bar = m_Hpk[:32]
    r_seed = m_Hpk[32:36]  # 4 bytes para la semilla
    
    m_ints = [int.from_bytes(m[i*4:(i+1)*4], 'big') % q for i in range(k)]
    m_vector = np.array(m_ints, dtype=int).reshape((k, 1))
    
    c, _, _, _ = encrypt(pk, m_vector, r_seed)
    u, v = c
    c_bytes = u.tobytes() + v.tobytes()
    H_c = hashlib.sha3_256(c_bytes).digest()
    K = hashlib.sha3_256(K_bar + H_c).digest()
    return c, K

---
# 5. Algoritmo de desencapsulado

El objetivo del desencapsulado es que el receptor recupere la clave compartida $K$ a partir de un mensaje cifrado $c$. Para ello, el receptor usa la clave secreta $sk$ y realiza una serie de pasos de verificación para asegurarse de que el mensaje cifrado no ha sido alterado.

1. **Cálculo del valor $h$:**  
   El receptor comienza calculando un valor $h$ a partir de su clave secreta $sk$.  
   Este valor $h$ se deriva utilizando parámetros específicos como $k$ (tamaño de la matriz) y $n$ (número de dimensiones), además de una constante $+32$. Este valor $h$ se utilizará en el proceso de verificación del mensaje cifrado.

2. **Cálculo del valor $z$:**  
   Luego, se calcula el segundo valor, $z$, que también es una combinación de la clave secreta $sk$ y los parámetros $k$ y $n$, pero con la constante $+64$.  
   Este valor $z$ es un valor intermedio que se utilizará en caso de que el mensaje cifrado haya sido alterado.

3. **Descifrado del mensaje:**  
   A continuación, el receptor utiliza su clave secreta $sk$ para descifrar el mensaje $c$ utilizando el algoritmo de descifrado $D(sk,c)$ (Algoritmo 3), obteniendo el mensaje $m ̃$.

4. **Generación de la clave compartida $K ̅$ y valor $r'$:**  
   Con el mensaje recuperado $m ̃$ y el valor $h$ calculado en el primer paso, el receptor genera una clave compartida $K ̅$ y un valor $r'$ mediante la función hash $G(m ̃||h)$.  
   - $K ̅$ es la clave compartida generada a partir de $m ̃$ y $h$.  
   - $r'$ es un valor adicional utilizado en el siguiente paso para garantizar la integridad.  
   La función hash $G$ asegura que el proceso sea seguro y único. Debido a que $G$ es una función hash criptográfica, tanto $K ̅$ como $r'$ son determinísticos pero difíciles de predecir sin conocer $m ̃$ y $h$.

5. **Generación del mensaje cifrado $c'$:**  
   Después de obtener $K ̅$ y $r'$, el receptor genera un nuevo mensaje cifrado $c'$ utilizando la función de cifrado $E(pk, m ̃, r')$.

6. **Comparación de los mensajes cifrados:**  
   El receptor compara el mensaje cifrado $c'$ con el mensaje cifrado $c$ que recibió.  
   - Si $c'$ es igual a $c$, significa que el proceso fue correcto y que $K ̅$ es la clave compartida correcta.  
   - Si la verificación falla, el receptor utiliza el valor alternativo $z$ para generar la clave $K$ de forma diferente. Esta clave será totalmente inútil para un atacante.

7. **Generación de la clave final $K$:**  
   A continuación, el receptor genera la clave final $K$ mediante la función de derivación de claves (KDF).  
   - Se concatena la clave derivada $K ̅$ con el valor hash $H(c)$ de $c$, lo que ayuda a asegurar la integridad y autenticidad del mensaje cifrado.

Si la verificación fue exitosa, el receptor tendrá la clave final $K$, que se utilizará para la comunicación segura.


In [7]:
# Algoritmo 6: Desencapsulado KEM
def decapsulate(sk, c):
    sk_prime_len = 4 * k
    pk_len = 4*k + 16
    h_start = sk_prime_len + pk_len
    h = sk[h_start:h_start+32]
    z = sk[h_start+32:h_start+64]
    
    s = np.frombuffer(sk[:sk_prime_len], dtype=np.int32).reshape((k,1))
    pk = (np.frombuffer(sk[sk_prime_len:sk_prime_len+4*k], dtype=np.int32).reshape((k,1)), 
          sk[sk_prime_len+4*k:sk_prime_len+pk_len])
    
    m_tilde = decrypt(s, c)
    m_tilde_bytes = m_tilde.tobytes()
    m_tilde_h = m_tilde_bytes + h
    G_output = hashlib.sha3_256(m_tilde_h).digest()
    K_bar_prime = G_output[:32]
    r_prime_seed = G_output[32:]
    
    c_prime, _, _, _ = encrypt(pk, m_tilde, r_prime_seed)
    u, v = c
    c_bytes = u.tobytes() + v.tobytes()
    u_prime, v_prime = c_prime
    c_prime_bytes = u_prime.tobytes() + v_prime.tobytes()
    
    H_c = hashlib.sha3_256(c_bytes).digest()
    if c_bytes == c_prime_bytes:
        return hashlib.sha3_256(K_bar_prime + H_c).digest()
    else:
        return hashlib.sha3_256(z + H_c).digest()

---
# 6. Ejemplo de uso

A continuación se muestra un ejemplo de uso utilizando los tres algoritmos:

In [13]:
# Ejemplo de uso
if __name__ == "__main__":
    # Generar claves KEM
    pk, sk = key_gen_G()
    
    # Encapsular
    ciphertext, K_encap = encapsulate(pk)
    
    # Desencapsular
    K_decap = decapsulate(sk, ciphertext)
    
    print("Clave encapsulada:", K_encap.hex())
    print("Clave desencapsulada:", K_decap.hex())
    print("Coinciden?", K_encap == K_decap)

Clave encapsulada: 1bd0d04d4fc36f6cfa660644e6c3cf991d1281deec2869b0fbe898dd982245e9
Clave desencapsulada: e9c72a63a3bd519e19d51d04802be4a70c5120e6a16ef8510d76d6fd2b7383d6
Coinciden? False
